In [1]:
# ==========================================
# ✅ 1) Install TensorFlow versi terbaru yg kompatibel
# ==========================================
!pip install -q tensorflow==2.19.0


# ==========================================
# ✅ 2) Imports
# ==========================================
import os
import re
import random
import tarfile
import numpy as np
import tensorflow as tf
from glob import glob
from pathlib import Path
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, optimizers, losses, metrics
from tensorflow.keras.applications import MobileNetV2

# ==========================================
# ✅ 3) Mount Google Drive
# ==========================================
from google.colab import drive
drive.mount('/content/drive')

RAW_DATA_DIR = "/content/drive/MyDrive/UTKface_inthewild"   # Folder yang berisi .tar.gz

# ==========================================
# ✅ 4) Ekstraksi TAR.GZ ke folder lokal (bukan drive)
# ==========================================
def extract_all_tar_gz(src_folder, dst_folder="/content/UTK_extract"):
    os.makedirs(dst_folder, exist_ok=True)
    tar_files = [f for f in os.listdir(src_folder) if f.endswith(".tar.gz")]

    # Jika sudah diekstrak sebelumnya, skip
    if len(glob(os.path.join(dst_folder, "*.jpg"))) > 1000:
        print("✅ File sudah diekstrak sebelumnya, skip.")
        return dst_folder

    print(f"🔍 Menemukan {len(tar_files)} file .tar.gz → ekstraksi ke: {dst_folder}\n")

    for file in tar_files:
        file_path = os.path.join(src_folder, file)
        print(f"📦 Extracting: {file_path}")
        with tarfile.open(file_path, "r:gz") as tar:
            tar.extractall(path=dst_folder)
        print("✅ Selesai:", file)

    print("\n🔥 Semua file berhasil diekstrak!")
    return dst_folder

DATA_DIR = extract_all_tar_gz(RAW_DATA_DIR)

# ==========================================
# ✅ 5) Filename regex parser (ambil age)
# ==========================================
FNAME_RE = re.compile(r"^(\d{1,3})[ _-]?(\d)[ _-]?(\d)_.*\.(jpg|jpeg|png)$", re.IGNORECASE)

def parse_age_from_filename(fname):
    base = os.path.basename(fname)
    m = FNAME_RE.match(base)
    if m:
        return int(m.group(1))
    return None

# ==========================================
# ✅ 6) Kumpulkan list image + age labels
# ==========================================
def build_image_list(data_dir):
    files = glob(os.path.join(data_dir, "**/*.jpg"), recursive=True)
    files += glob(os.path.join(data_dir, "**/*.png"), recursive=True)

    items = []
    for f in files:
        age = parse_age_from_filename(f)
        if age is not None:
            items.append((f, age))
    return items

items = build_image_list(DATA_DIR)
print("Total images parsed:", len(items))
if len(items) == 0:
    raise RuntimeError("❌ Tidak menemukan file gambar yang sesuai format UTKFace!")

# ==========================================
# ✅ 7) Train/Val/Test Split
# ==========================================
random.shuffle(items)
paths, ages = zip(*items)
paths = np.array(paths)
ages = np.array(ages, dtype=np.float32)

p_train, p_val, p_test = 0.8, 0.1, 0.1
X_train, X_temp, y_train, y_temp = train_test_split(paths, ages, test_size=(1-p_train), random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=(p_test/(p_val+p_test)), random_state=42)

print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")

# ==========================================
# ✅ 8) TF Dataset Pipeline
# ==========================================
IMG_SIZE = (160,160)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

def load_and_preprocess(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, IMG_SIZE)
    return img, label

def make_dataset(paths, labels, training=False):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if training: ds = ds.shuffle(8000)
    ds = ds.map(load_and_preprocess, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds

train_ds = make_dataset(X_train, y_train, training=True)
val_ds   = make_dataset(X_val,   y_val)
test_ds  = make_dataset(X_test,  y_test)

# ==========================================
# ✅ 9) Model - MobileNetV2 + Regression Head
# ==========================================
def build_model():
    base = MobileNetV2(input_shape=(*IMG_SIZE,3), include_top=False, weights="imagenet", pooling="avg")
    base.trainable = False
    inputs = layers.Input(shape=(*IMG_SIZE,3))
    x = base(inputs, training=False)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(1, activation='linear')(x)
    return models.Model(inputs, outputs)

model = build_model()
model.compile(optimizer=optimizers.Adam(1e-3),
              loss=losses.MeanAbsoluteError(),
              metrics=[metrics.MeanAbsoluteError(), metrics.MeanSquaredError()])
model.summary()

# ==========================================
# ✅ 10) Training callbacks
# ==========================================
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
ckpt_path = "age_estimator_mobilenetv2.h5"

callbacks = [
    EarlyStopping(monitor='val_mean_absolute_error', patience=5, restore_best_weights=True),
    ModelCheckpoint(ckpt_path, save_best_only=True, monitor='val_mean_absolute_error'),
    ReduceLROnPlateau(monitor='val_mean_absolute_error', factor=0.5, patience=3)
]

# ==========================================
# ✅ 11) Train Base Model
# ==========================================
history = model.fit(train_ds, validation_data=val_ds, epochs=20, callbacks=callbacks)

# ==========================================
# ✅ 12) Fine-Tune (Unfreeze last layers)
# ==========================================
base = model.layers[1]
base.trainable = True
for layer in base.layers[:-30]:
    layer.trainable = False

model.compile(optimizer=optimizers.Adam(1e-5),
              loss=losses.MeanAbsoluteError(),
              metrics=[metrics.MeanAbsoluteError(), metrics.MeanSquaredError()])

history_ft = model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=callbacks)

# ==========================================
# ✅ 13) Evaluate on Test
# ==========================================
result = model.evaluate(test_ds)
print("Test results (loss, MAE, MSE):", result)

# ==========================================
# ✅ 14) Save Model
# ==========================================
model.save("final_age_model.keras")


# ==========================================
# ✅ 15) Inference Function
# ==========================================
from PIL import Image

def preprocess_for_inference(img_path):
    img = Image.open(img_path).convert("RGB")
    img = img.resize(IMG_SIZE)
    arr = np.array(img).astype(np.float32) / 255.0
    return np.expand_dims(arr, axis=0)

def predict_age(img_path, model):
    x = preprocess_for_inference(img_path)
    pred = float(model.predict(x)[0][0])
    return max(0.0, min(116.0, pred))  # clamp

# Example:
# age = predict_age("/content/sample.jpg", model)
# print("Predicted age:", age)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔍 Menemukan 3 file .tar.gz → ekstraksi ke: /content/UTK_extract

📦 Extracting: /content/drive/MyDrive/UTKface_inthewild/part1.tar.gz


/tmp/ipython-input-3183109865.py:48: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=dst_folder)


✅ Selesai: part1.tar.gz
📦 Extracting: /content/drive/MyDrive/UTKface_inthewild/part2.tar.gz
✅ Selesai: part2.tar.gz
📦 Extracting: /content/drive/MyDrive/UTKface_inthewild/part3.tar.gz
✅ Selesai: part3.tar.gz

🔥 Semua file berhasil diekstrak!
Total images parsed: 24105
Train: 19284 | Val: 2410 | Test: 2411


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_160            │ (None, 1280)           │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

603/603 ━━━━━━━━━━━━━━━━━━━━ 663s 1s/step - loss: 7.8329 - mean_absolute_error: 7.8329 - mean_squared_error: 121.9304 - val_loss: 8.2914 - val_mean_absolute_error: 8.2914 - val_mean_squared_error: 142.8862 - learning_rate: 1.0000e-05
Epoch 6/10
603/603 ━━━━━━━━━━━━━━━━━━━━ 0s 940ms/step - loss: 7.4947 - mean_absolute_error: 7.4947 - mean_squared_error: 113.6644

603/603 ━━━━━━━━━━━━━━━━━━━━ 621s 1s/step - loss: 7.4947 - mean_absolute_error: 7.4947 - mean_squared_error: 113.6637 - val_loss: 8.2085 - val_mean_absolute_error: 8.2085 - val_mean_squared_error: 140.1172 - learning_rate: 1.0000e-05
Epoch 7/10
603/603 ━━━━━━━━━━━━━━━━━━━━ 0s 953ms/step - loss: 7.1990 - mean_absolute_error: 7.1990 - mean_squared_error: 104.7891

603/603 ━━━━━━━━━━━━━━━━━━━━ 657s 1s/step - loss: 7.1991 - mean_absolute_error: 7.1991 - mean_squared_error: 104.7926 - val_loss: 8.1052 - val_mean_absolute_error: 8.1052 - val_mean_squared_error: 137.0047 - learning_rate: 1.0000e-05
Epoch 8/10
603/603 ━━━━━━━━━━━━━━━━━━━━ 0s 946ms/step - loss: 6.8956 - mean_absolute_error: 6.8956 - mean_squared_error: 96.2521

603/603 ━━━━━━━━━━━━━━━━━━━━ 625s 1s/step - loss: 6.8957 - mean_absolute_error: 6.8957 - mean_squared_error: 96.2552 - val_loss: 8.0583 - val_mean_absolute_error: 8.0583 - val_mean_squared_error: 135.5340 - learning_rate: 1.0000e-05
Epoch 9/10
603/603 ━━━━━━━━━━━━━━━━━━━━ 619s 1s/step - loss: 6.6170 - mean_absolute_error: 6.6170 - mean_squared_error: 87.7816 - val_loss: 8.0591 - val_mean_absolute_error: 8.0591 - val_mean_squared_error: 135.5485 - learning_rate: 1.0000e-05
Epoch 10/10
603/603 ━━━━━━━━━━━━━━━━━━━━ 0s 939ms/step - loss: 6.6205 - mean_absolute_error: 6.6205 - mean_squared_error: 88.4871

603/603 ━━━━━━━━━━━━━━━━━━━━ 623s 1s/step - loss: 6.6205 - mean_absolute_error: 6.6205 - mean_squared_error: 88.4857 - val_loss: 7.9754 - val_mean_absolute_error: 7.9754 - val_mean_squared_error: 133.1622 - learning_rate: 1.0000e-05
76/76 ━━━━━━━━━━━━━━━━━━━━ 52s 677ms/step - loss: 7.8616 - mean_absolute_error: 7.8616 - mean_squared_error: 121.5580
Test results (loss, MAE, MSE): [7.976768493652344, 7.976768493652344, 126.09506225585938]


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install -q --upgrade numpy==2.1.3
!pip install -q --upgrade tensorflow==2.19.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 614.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 11.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.1.3 which is incompatible.


In [4]:
import tensorflow as tf

# Load keras model
model = tf.keras.models.load_model("/content/final_age_model.keras")

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save file
with open("age_model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Conversion done, file saved: age_model.tflite")


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 36 variables whereas the saved optimizer has 70 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Saved artifact at '/tmp/tmpepvggcce'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 160, 160, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  134131246860048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134131131294800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134131131294032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134131131295952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134131131295376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134131131295184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134131131297488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134131131294416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134131131296912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134131131298448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1341311312965